In [1]:
from read_data import *
from hyper import *
from alignment import *
from datasets.preprecossing import *
from core import *

In [ ]:
cell_path1 = './datas/pre3/sample1_1.h5'
folder_path1 = "./datas/10-19/d1/"
radius1 = 147.63807097087118
c1= 0.001
epoches1 = 15

In [ ]:
cell_path2 = './datas/pre3/sample2_1.h5'
folder_path2 = "./datas/10-19/d2/"
radius2 = 149.58609561052123
c2= 0.001
epoches2 =15

In [ ]:
adata = sc.read_h5ad(cell_path1)
datas = []
for i in adata.obs['celltype'].unique():
    datas.append(adata[adata.obs['celltype']==i])
for i in datas:
    preprocessing_cluster(i,N_pcs=50,resolution=0.2)
adata.obs['leiden']=['0' for i in range(len(adata))]
for i in datas:
    for j in range(len(i)):
        name = i.obs.index[j]
        leiden_num = i.obs['leiden'][j]
        index = adata.obs.index.get_loc(i.obs.index[j])
        adata.obs['leiden'][index] = adata.obs['celltype'][index] + '_' + leiden_num
copys = adata.to_df()
copys['leiden']=adata.obs['leiden']
copys=copys.groupby('leiden').mean()
copys.to_csv(folder_path1+"datas.data",header=None);
copys.to_csv(folder_path1+"datas.csv");

In [ ]:
def julei(cell_path,folder_path,resolution=0.2):
    adata = sc.read_h5ad(cell_path)
    datas = []
    for i in adata.obs['celltype'].unique():
        datas.append(adata[adata.obs['celltype']==i])
    for i in datas:
        preprocessing_cluster(i,N_pcs=50,resolution=resolution)
    adata.obs['leiden']=['0' for i in range(len(adata))]
    for i in datas:
        for j in range(len(i)):
            name = i.obs.index[j]
            leiden_num = i.obs['leiden'][j]
            index = adata.obs.index.get_loc(i.obs.index[j])
            adata.obs['leiden'][index] = adata.obs['celltype'][index] + '_' + leiden_num
    copys = adata.to_df()
    copys['leiden']=adata.obs['leiden']
    copys=copys.groupby('leiden').mean()
    copys.to_csv(folder_path+"datas.data",header=None);
    copys.to_csv(folder_path+"datas.csv");
    return copys


In [ ]:
for i in range(1,21):
    resolution = i*0.05;
    copys1 = julei(cell_path1,folder_path1,resolution)
    copys2 = julei(cell_path2,folder_path2,resolution)
    get_Hyper_tree(folder_path1+'datas.data',1,copys1.shape[1]+1,0,epoches1,save_path=folder_path1,c=0)
    get_Hyper_tree(folder_path2+'datas.data',1,copys2.shape[1]+1,0,epoches2,save_path=folder_path2,c=0)
    nodes1,n1 = build_hyper_tree(folder_path1)
    nodes2,n2 = build_hyper_tree(folder_path2)

    merge_list1 = [];
    merge_list2 = [];
    nodes1[0] = search_tree(nodes1[0],c1,merge_list1)
    nodes2[0] = search_tree(nodes2[0],c2,merge_list2)
    meta_list1 = pd.read_csv(folder_path1+"datas.csv");
    meta_list1 = meta_list1['leiden'].tolist()
    meta_list2 = pd.read_csv(folder_path2+"datas.csv");
    meta_list2 = meta_list2['leiden'].tolist()
    for i in range(len(nodes1)):
        if(int(nodes1[i].name)<len(meta_list1)):
            nodes1[i].name= nodes1[i].name +'_'+ meta_list1[int(nodes1[i].name)];
            
    for i in range(len(nodes2)):
        if(int(nodes2[i].name)<len(meta_list2)):
            nodes2[i].name= nodes2[i].name +'_'+ meta_list2[int(nodes2[i].name)];  

    T=tree_alignment(nodes1[0],nodes2[0],1);
    minn = T.run_alignment();
    T.show_ans();
    ans = T.get_ans()
    G=show_graph(ans,nodes1[0],nodes2[0]);
    c=0;z=0
    for i,j in ans:
        i=int(i.split('_')[0])
        j=int(j.split('_')[0])
        if(i<len(meta_list1) and j <len(meta_list2)):
            c+=1
            if(meta_list1[i].split('_')[0]==meta_list2[j].split('_')[0]):
                z+=1;
    print('correct alignment rate:{}'.format(z/c))
    print("resolution={}".format(resolution))


In [ ]:
T=tree_alignment(nodes1[0],nodes2[0],1);
minn = T.run_alignment();
T.show_ans();
ans = T.get_ans()
G=show_graph(ans,nodes1[0],nodes2[0]);
c=0;z=0
for i,j in ans:
    i=int(i.split('_')[0])
    j=int(j.split('_')[0])
    if(i<len(meta_list1) and j <len(meta_list2)):
        c+=1
        if(meta_list1[i].split('_')[0]==meta_list2[j].split('_')[0]):
            z+=1;
print('correct alignment rate:{}'.format(z/c))

In [ ]:
meta_list2[j]

The alignment edges list is [('32', '32'), ('31', '30'), ('29', '28'), ('28', '27'), ('26', '26'), ('24', '25'), ('23', '21'), ('21', '20'), ('9_FPP_1', '9_FPP_1'), ('6_Epen1_3', '5_Epen1_2'), ('8_FPP_0', '12_P_FPP_0'), ('12_P_FPP_2', '6_Epen1_3'), ('18', '24'), ('17', '17'), ('2_DA_2', '2_DA_2'), ('0_DA_0', '0_DA_0'), ('1_DA_1', '8_FPP_0'), ('19', '23'), ('11_P_FPP_1', '1_DA_1'), ('3_Epen1_0', '14_Sert_0'), ('27', '16_Sert_2'), ('15_Sert_2', '15_Sert_1'), ('30', '29'), ('25', '22'), ('7_Epen1_4', '7_Epen1_4'), ('5_Epen1_2', '4_Epen1_1'), ('4_Epen1_1', '3_Epen1_0')]
### resolution 0.2
### correct alignment rate:0.6153846153846154

The alignment edges list is [('18', '18'), ('15', '17'), ('14', '16'), ('13', '14'), ('11', '11'), ('10', '10'), ('8_P_FPP_1', '8_P_FPP_0'), ('0_DA_0', '6_FPP_0'), ('5_FPP_0', '2_Epen1_0'), ('1_DA_1', '9_Sert_0'), ('6_FPP_1', '7_FPP_1'), ('9_Sert_0', '1_DA_1'), ('16', '13'), ('12', '12'), ('3_Epen1_1', '5_Epen1_3'), ('2_Epen1_0', '4_Epen1_2'), ('4_Epen1_2', '3_Epen1_1')]

correct alignment rate:0.5555555555555556

resolution 0.1

The alignment edges list is [('44', '44'), ('43', '42'), ('42', '41'), ('39', '31'), ('8_Epen1_4', '19_Sert_1'), ('6_Epen1_2', '21_Sert_3'), ('35', '20_Sert_2'), ('40', '40'), ('38', '37'), ('36', '35'), ('31', '30'), ('30', '28'), ('27', '26'), ('24', '25'), ('10_FPP_1', '10_FPP_0'), ('7_Epen1_3', '0_DA_0'), ('23', '23'), ('16_P_FPP_2', '15_P_FPP_1'), ('9_FPP_0', '13_FPP_3'), ('12_FPP_3', '16_P_FPP_2'), ('15_P_FPP_1', '12_FPP_2'), ('26', '34'), ('1_DA_1', '18_Sert_0'), ('0_DA_0', '3_DA_3'), ('29', '33'), ('21_Sert_3', '2_DA_2'), ('18_Sert_0', '1_DA_1'), ('34', '39'), ('32', '27'), ('17_P_FPP_3', '17_P_FPP_3'), ('14_P_FPP_0', '14_P_FPP_0'), ('11_FPP_2', '11_FPP_1'), ('37', '43'), ('22_Sert_4', '8_Epen1_4'), ('20_Sert_2', '38')]

correct alignment rate:0.4375

resolution 0.3


The alignment edges list is [('66', '64'), ('63', '62'), ('60', '58'), ('48', '50'), ('34', '35'), ('30_Sert_4', '12_Epen1_6'), ('28_Sert_2', '13_Epen1_7'), ('32_Sert_6', '8_Epen1_2'), ('29_Sert_3', '7_Epen1_1'), ('31_Sert_5', '59'), ('61', '63'), ('59', '61'), ('58', '60'), ('54', '57'), ('53', '54'), ('52', '49'), ('51', '43'), ('47', '34'), ('17_FPP_3', '15_FPP_1'), ('12_Epen1_6', '5_DA_5'), ('41', '40'), ('37', '38'), ('36', '33'), ('18_FPP_4', '14_FPP_0'), ('10_Epen1_4', '11_Epen1_5'), ('16_FPP_2', '16_FPP_2'), ('22_P_FPP_1', '21_P_FPP_0'), ('35', '48'), ('23_P_FPP_2', '3_DA_3'), ('15_FPP_1', '18_FPP_4'), ('50', '53'), ('49', '51'), ('43', '46'), ('33_Sert_7', '20_FPP_6'), ('27_Sert_1', '23_P_FPP_2'), ('42', '41'), ('3_DA_3', '17_FPP_3'), ('2_DA_2', '22_P_FPP_1'), ('19_FPP_5', '6_Epen1_0'), ('26_Sert_0', '25_P_FPP_4'), ('1_DA_1', '27_Sert_0'), ('45', '10_Epen1_4'), ('57', '56'), ('6_Epen1_0', '30_Sert_3'), ('46', '55'), ('14_FPP_0', '32_Sert_5'), ('21_P_FPP_0', '29_Sert_2')]

correct alignment rate:0.2727272727272727

0.5

In [ ]:

resolution = 1*0.05;
copys1 = julei(cell_path1,folder_path1,resolution)
copys2 = julei(cell_path2,folder_path2,resolution)
get_Hyper_tree(folder_path1+'datas.data',1,copys1.shape[1]+1,0,epoches1,save_path=folder_path1,c=0)
get_Hyper_tree(folder_path2+'datas.data',1,copys2.shape[1]+1,0,epoches2,save_path=folder_path2,c=0)
nodes1,n1 = build_hyper_tree(folder_path1)
nodes2,n2 = build_hyper_tree(folder_path2)

merge_list1 = [];
merge_list2 = [];
nodes1[0] = search_tree(nodes1[0],c1,merge_list1)
nodes2[0] = search_tree(nodes2[0],c2,merge_list2)
meta_list1 = pd.read_csv(folder_path1+"datas.csv");
meta_list1 = meta_list1['leiden'].tolist()
meta_list2 = pd.read_csv(folder_path2+"datas.csv");
meta_list2 = meta_list2['leiden'].tolist()
for i in range(len(nodes1)):
    if(int(nodes1[i].name)<len(meta_list1)):
        nodes1[i].name= nodes1[i].name +'_'+ meta_list1[int(nodes1[i].name)];
        
for i in range(len(nodes2)):
    if(int(nodes2[i].name)<len(meta_list2)):
        nodes2[i].name= nodes2[i].name +'_'+ meta_list2[int(nodes2[i].name)];  

T=tree_alignment(nodes1[0],nodes2[0],1);
minn = T.run_alignment();
T.show_ans();
ans = T.get_ans()
G=show_graph(ans,nodes1[0],nodes2[0]);
c=0;z=0
for i,j in ans:
    i=int(i.split('_')[0])
    j=int(j.split('_')[0])
    if(i<len(meta_list1) and j <len(meta_list2)):
        c+=1
        if(meta_list1[i].split('_')[0]==meta_list2[j].split('_')[0]):
            z+=1;
print('correct alignment rate:{}'.format(z/c))
print("resolution={}".format(resolution))


In [ ]:
G.save_fig(folder_path1+'alignment.png')


In [ ]:

resolution = 1.375*0.05;
copys1 = julei(cell_path1,folder_path1,resolution)
copys2 = julei(cell_path2,folder_path2,resolution)
get_Hyper_tree(folder_path1+'datas.data',1,copys1.shape[1]+1,0,epoches1,save_path=folder_path1,c=0)
get_Hyper_tree(folder_path2+'datas.data',1,copys2.shape[1]+1,0,epoches2,save_path=folder_path2,c=0)
nodes1,n1 = build_hyper_tree(folder_path1)
nodes2,n2 = build_hyper_tree(folder_path2)

merge_list1 = [];
merge_list2 = [];
nodes1[0] = search_tree(nodes1[0],c1,merge_list1)
nodes2[0] = search_tree(nodes2[0],c2,merge_list2)
meta_list1 = pd.read_csv(folder_path1+"datas.csv");
meta_list1 = meta_list1['leiden'].tolist()
meta_list2 = pd.read_csv(folder_path2+"datas.csv");
meta_list2 = meta_list2['leiden'].tolist()
for i in range(len(nodes1)):
    if(int(nodes1[i].name)<len(meta_list1)):
        nodes1[i].name= nodes1[i].name +'_'+ meta_list1[int(nodes1[i].name)];
        
for i in range(len(nodes2)):
    if(int(nodes2[i].name)<len(meta_list2)):
        nodes2[i].name= nodes2[i].name +'_'+ meta_list2[int(nodes2[i].name)];  

T=tree_alignment(nodes1[0],nodes2[0],1);
minn = T.run_alignment();
T.show_ans();
ans = T.get_ans()
G=show_graph(ans,nodes1[0],nodes2[0]);
c=0;z=0
for i,j in ans:
    i=int(i.split('_')[0])
    j=int(j.split('_')[0])
    if(i<len(meta_list1) and j <len(meta_list2)):
        c+=1
        if(meta_list1[i].split('_')[0]==meta_list2[j].split('_')[0]):
            z+=1;
print('correct alignment rate:{}'.format(z/c))
print("resolution={}".format(resolution))


In [ ]:
G.save_fig(folder_path1+'alignment.png')
